# The best performing model so far on asl100

In [79]:
import torch
import test
from configs import Config
from torchvision.transforms import v2
from video_dataset import VideoDataset
import os
from torch.utils.data import DataLoader
from models.pytorch_r3d import Resnet3D18_basic
import json
import utils

In [80]:
torch.manual_seed(42)
model = 'r3d18'
split = 'asl100'
exp_no = '6'.zfill(3)


root = '../data/WLASL2000'
labels=f'./preprocessed/labels/{split}'
output=f'runs/{split}/{model}_exp{exp_no}'
config_path = f'./configfiles/{split}/{model}_{exp_no}.ini'
configs = Config(config_path)
model_dict='best.pth'
classes_path = './wlasl_class_list.json'
label_suffix='_fixed_frange_bboxes_len.json'
title = f'''Testting {model} on split {split} 
              Experiment no: {exp_no} 
              Raw videos at: {root}
              Labels at: {labels}
              Saving files to: {output}
              Model weights: {model_dict}
              {str(configs)}
              \n
          '''
print(title)

Testting r3d18 on split asl100 
              Experiment no: 006 
              Raw videos at: ../data/WLASL2000
              Labels at: ./preprocessed/labels/asl100
              Saving files to: runs/asl100/r3d18_exp006
              Model weights: best.pth
              Config:
		Model: <class 'models.pytorch_r3d.Resnet3D18_basic'>
		Weights Path: 
		Frozen layers: []
		Scheduler: t_max=100, eta_min=1e-05
		Training: bs=6, steps=64000, ups=1
		Optimizer: lr=0.001, eps=0.001, wd=1e-07
		Backbone: lr=1e-05, wd=0.0001
		Classifier: lr=0.001, wd=0.0001
              

          


In [81]:

# test.run_test_r3d18_1(configs,
#                       root=root,
#                       output=output,
#                       labels=labels,
#                       model_dict=model_dict,
#                       save=True)

In [82]:
#setup transforms
base_mean = [0.43216, 0.394666, 0.37645]
base_std = [0.22803, 0.22145, 0.216989]

r3d18_final = v2.Compose([
  v2.Lambda(lambda x: x.float() / 255.0),
  # v2.Lambda(lambda x: vt.normalise(x, base_mean, base_std)),
  v2.Normalize(mean=base_mean, std=base_std),
  v2.Lambda(lambda x: x.permute(1,0,2,3)) 
])

test_transforms = v2.Compose([v2.CenterCrop(224),
                              r3d18_final])

#setup data
test_instances = os.path.join(labels, 'test_instances_fixed_frange_bboxes_len.json')
test_classes = os.path.join(labels, 'test_classes_fixed_frange_bboxes_len.json')

test_set = VideoDataset(root, test_instances, test_classes,
                        transforms=test_transforms, num_frames=32)
test_loader = DataLoader(test_set, batch_size=1,shuffle=False,
                          num_workers=0)
num_classes = len(set(test_set.classes))
# print(num_classes)

In [83]:


# #setup model
# r3d18 = Resnet3D18_basic(num_classes=num_classes, drop_p=configs.drop_p,)
# r3d18_dict = torch.load(os.path.join(output,'checkpoints', model_dict)) #future warning, use weights_only=True (security stuff if you dont know the file)
# # print(r3d18_dict)
# r3d18.load_state_dict(r3d18_dict)
# r3d18.cuda()
# r3d18.eval()

# accuracy, class_report = test.test_model(r3d18, test_loader)
# print(f'Accuracy score: {accuracy}')

In [84]:
# test.plot_heatmap(class_report, classes_path)

In [85]:
# test.plot_bar_graph(class_report, classes_path)

## Whats weird is that the classes can almost be split into distinct difficulty groups

The model seems to have fully learned some signs, while completely failing to learn others. The rest have stagering degrees of recognition


### Lets see if we can visualise what it sees for the sign 'thursday'


In [86]:

with open(classes_path, 'r') as f:
  class_list = json.load(f)
print(class_list[99])

thursday


In [87]:
print(test_set.classes[-10:])

['right', 'same', 'same', 'same', 'son', 'son', 'tell', 'tell', 'thursday', 'thursday']


In [88]:
print(len(test_set))

258


In [89]:
item = test_set.data[256]

In [90]:
def print_dict(dic):
  print("{")
  for key, value in dic.items():
    print(f"'{key}': {value}")
  print("}")  

In [91]:

frames= test_set.__manual_load__(item)
print_dict(item)

{
'label_num': 99
'frame_end': 46
'frame_start': 1
'video_id': 58359
'bbox': [51, 41, 212, 222]
}


In [92]:
print(len(frames))
print(type(frames))

32
<class 'torch.Tensor'>


In [93]:
print(frames.shape)

torch.Size([32, 3, 256, 256])


In [94]:
utils.watch_video(frames)
# utils.visualise_frames(frames, 32)

### Honestly quite difficult, let's look at another

In [95]:
item1 = test_set.data[257]
frames1= test_set.__manual_load__(item1)
print_dict(item1)
print(len(frames1))

{
'label_num': 99
'frame_end': 92
'frame_start': 0
'video_id': 70026
'bbox': [61, 30, 199, 235]
}
32


In [96]:
utils.watch_video(frames1)

In [97]:
print(item)

{'label_num': 99, 'frame_end': 46, 'frame_start': 1, 'video_id': '58359', 'bbox': [51, 41, 212, 222]}


### Let's gander at the originals

In [98]:
vid_path = os.path.join(root, f"{item['video_id']}.mp4")
frames_o = utils.load_rgb_frames_from_video(vid_path, item['frame_start'],
                                          item['frame_end'])
print(len(frames_o))
print(vid_path)

45
../data/WLASL2000/58359.mp4


In [99]:
utils.watch_video(frames_o)

In [100]:
vid_path1 = os.path.join(root, f"{item1['video_id']}.mp4")
frames_o1 = utils.load_rgb_frames_from_video(vid_path1, item1['frame_start'],
                                          item1['frame_end'])
print(len(frames_o1))
print(vid_path1)

92
../data/WLASL2000/70026.mp4


In [101]:
utils.watch_video(frames_o1)

### Okay cool they are not being distorted too much through loading, lets see after transforms

In [102]:
frames_t = test_transforms(frames)
frames_t = frames_t.permute(1,0,2,3) #just reuturn back to T C H W
frames_t = (frames_t * 255).clamp(0, 255).to(torch.uint8)

print(frames_t.dtype)
print(frames_t.shape)

torch.uint8
torch.Size([32, 3, 224, 224])


In [103]:
utils.watch_video(frames_t)

In [104]:
frames_t1 = test_transforms(frames1)
frames_t1 = frames_t1.permute(1,0,2,3) #just reuturn back to T C H W
frames_t1 = (frames_t1 * 255).clamp(0, 255).to(torch.uint8)

print(frames_t1.dtype)
print(frames_t1.shape)

torch.uint8
torch.Size([32, 3, 224, 224])


In [105]:
utils.watch_video(frames_t1)

### Normalising seems to have quite a significant effect on the videos appearence, but stil decernable

## Let's view some samples of this class from the training set

In [106]:
train_transforms, test_transforms = configs.get_transforms()
  
train_instances = os.path.join(labels, f'train_instances{label_suffix}')
train_classes = os.path.join(labels, f'train_classes{label_suffix}')

In [107]:
dataset = VideoDataset(root,train_instances, train_classes,
    transforms=train_transforms, num_frames=configs.num_frames)
dataloader = DataLoader(dataset, batch_size=1,
  shuffle=False, num_workers=0,pin_memory=False)
print(len(dataset))
num_classes = len(set(dataset.classes))
print(num_classes)

1442
100


In [108]:
print(dataset.classes[-13:])

['thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday', 'thursday']


In [109]:
titems = [dataset.data[i] for i in range(-1, -14, -1)]

for itm in titems:
  print_dict(itm)

{
'label_num': 99
'frame_end': 74
'frame_start': 1
'video_id': 58361
'bbox': [65, 24, 191, 237]
}
{
'label_num': 99
'frame_end': 71
'frame_start': 1
'video_id': 58360
'bbox': [45, 40, 205, 220]
}
{
'label_num': 99
'frame_end': 55
'frame_start': 1
'video_id': 58369
'bbox': [52, 37, 198, 225]
}
{
'label_num': 99
'frame_end': 43
'frame_start': 1
'video_id': 67306
'bbox': [54, 33, 200, 228]
}
{
'label_num': 99
'frame_end': 58
'frame_start': 1
'video_id': 66640
'bbox': [43, 40, 210, 220]
}
{
'label_num': 99
'frame_end': 64
'frame_start': 1
'video_id': 66639
'bbox': [42, 39, 210, 219]
}
{
'label_num': 99
'frame_end': 77
'frame_start': 1
'video_id': 66638
'bbox': [51, 34, 204, 226]
}
{
'label_num': 99
'frame_end': 72
'frame_start': 1
'video_id': 66637
'bbox': [49, 35, 205, 223]
}
{
'label_num': 99
'frame_end': 49
'frame_start': 1
'video_id': 58367
'bbox': [44, 28, 210, 255]
}
{
'label_num': 99
'frame_end': 33
'frame_start': 1
'video_id': 58366
'bbox': [53, 30, 202, 255]
}
{
'label_num': 99
'f

In [112]:
for itm in titems:
  tframes = dataset.__manual_load__(itm)
  utils.watch_video(tframes)

In [113]:
#with transforms
for itm in titems:
  tframes = dataset.__manual_load__(itm)
  tframes = test_transforms(tframes)
  tframes = tframes.permute(1,0,2,3) #just reuturn back to T C H W
  tframes = (tframes * 255).clamp(0, 255).to(torch.uint8)
  utils.watch_video(tframes)


### This does at least make me think I should be trying the effect of colour jitter